# Advanced Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Pandas is one of the most popular tools in Python for data analytics.  It contains data structures and data manipulation tools designed to make data cleaning and analysis fast and easy.

In this tutorial, we will play with a dataset from kaggle to demonstrate Pandas' basic operations.  The dataset is [Trending YouTube Video Statistics](https://www.kaggle.com/datasnaek/youtube-new).  For simplicity, we will work with only US dataset ([USvideos.csv](https://www.kaggle.com/datasnaek/youtube-new?select=USvideos.csv) and [US_category_id.json](https://www.kaggle.com/datasnaek/youtube-new?select=US_category_id.json)).

We will continue our lessons for advanced pandas.  This includes data wrangling and groupby operations.

In [ ]:
import pandas as pd
import numpy as np

## Youtube Trending Data Exploration

### Downloading data files from shared drive (optional for Colab)

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://www.dropbox.com/s/7u5j5uei57yyksh/data.tgz?dl=0 -O data.tgz
    !tar -xzvf data.tgz

### Read input from a data file into dataframe

In [ ]:
vdo_df = pd.read_csv('data/USvideos.csv')

### Remove Duplicates

Dataframe may contain some duplicate rows.

In [ ]:
vdo_df.drop_duplicates(inplace=True)

In [ ]:
vdo_df.shape

### Additional Data Preparation

In [ ]:
vdo_df['trending_dt'] = pd.to_datetime(vdo_df.trending_date, format='%y.%d.%m', errors='ignore', utc=True)

# Advanced Pandas Operations
When we deal with complex data and analysis, we usually have to to perform data wrangling.  In addition, groupby operations are usually required.

## Data Wrangling
Data contained in pandas objects can be combined together in a number of ways:
- *pandas.merge* connects rows in DataFrames based on one or more keys.  This is similar to SQL *join* operations
- *pandas.concat* concatenates or 'stacks' together objects along an axis.

### How each category trending in term of number of videos?
To answer this question, we will need to get category information from *US_category_id.json* file.

In [ ]:
vdo_df

In [ ]:
cat_df = pd.read_json('data/US_category_id.json')

In [ ]:
cat_df

US_category_id.json file seems to be more complicated than expected.  We will have to write a customized reader to get specific data from this json file.

In [ ]:
import json

In [ ]:
with open('data/US_category_id.json') as fd:
    cat = json.load(fd)

In [ ]:
cat

Extract only the id and snippet->title to be used for mapping

In [ ]:
cat_list = []
for d in cat['items']:
    cat_list.append((int(d['id']), d['snippet']['title']))

In [ ]:
cat_list

We can create a new dataframe from a list (or dict, etc.)

In [ ]:
cat_df = pd.DataFrame(cat_list, columns=['id', 'category'])

In [ ]:
cat_df

In [ ]:
vdo_df_withcat = vdo_df.merge(cat_df, left_on='category_id', right_on='id')

In [ ]:
vdo_df_withcat.columns

In [ ]:
vdo_df_withcat[['title', 'category_id', 'category']]

In [ ]:
vdo_df_withcat.category.value_counts()

**Merge function arguments**
- **left** <BR> DataFrame to be merged on the left side.
- **right** <BR> DataFrame to be merged on the right side.
- **how** <BR> One of 'inner', 'outer', 'left', or 'right'; defaults to 'inner'.
- **on** <BR> Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in left and right as the join keys.
- **left_on** <BR> Columns in left DataFrame to use as join keys.
- **right_on** <BR> Analogous to left_on for left DataFrame.
- **left_index** <BR> Use row index in left as its join key (or keys, if a MultiIndex).
- **right_index** <BR> Analogous to left_index.
- **sort** <BR> Sort merged data lexicographically by join keys; True by default (disable to get better performance in some cases on large datasets).
- **suffixes** <BR> Tuple of string values to append to column names in case of overlap; defaults to ('_x', '_y') (e.g., if 'data' in both DataFrame objects, would appear as 'data_x' and 'data_y' in result).
- **copy** <BR> If False, avoid copying data into resulting data structure in some exceptional cases; by default always copies.
- **indicator** <BR> Adds a special column _merge that indicates the source of each row; values will be 'left_only', 'right_only', or 'both' based on the origin of the joined data in each row.

In addition to merge function, we can also perform *concatenation* to combine 2 dataframes into one.  This is useful for merging other data.  For example, if we want to combine data from US and GB together, we can use concat.

In [ ]:
gb_vdo_df = pd.read_csv('data/GBvideos.csv')

In [ ]:
gb_vdo_df.shape

In [ ]:
vdo_df.shape

In [ ]:
pd.concat([vdo_df, gb_vdo_df], ignore_index=True, sort=True)

 **concat function arguments**

- **objs** <BR> List or dict of pandas objects to be concatenated; this is the only required argument
- **axis** <BR> Axis to concatenate along; defaults to 0 (along rows)
- **join** <BR> Either 'inner' or 'outer' ('outer' by default); whether to intersection (inner) or union (outer) together indexes along the other axes
- **join_axes** <BR> Specific indexes to use for the other nâ€“1 axes instead of performing union/intersection logic
- **keys** <BR> Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can either be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels)
- **levels** <BR> Specific indexes to use as hierarchical index level or levels if keys passed
- **names** <BR> Names for created hierarchical levels if keys and/or levels passed
- **verify_integrity** <BR> Check new axis in concatenated object for duplicates and raise exception if so; by default (False) allows duplicates
- **ignore_index** <BR> Do not preserve indexes along concatenation axis, instead producing a new range(total_length) index

##  Aggregation and Group Operations
Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.

### Group Operation Mechanics
Group operations can be described using the concepts of *split-apply-combine*.
- **The first stage** - data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object.
- **The second stage** - a function is applied to each group, producing a new value.
- **The third stage** - the results of all those function applications are combined into a result object.

In [ ]:
vdo_df_groupby_cat = vdo_df_withcat.groupby('category')

In [ ]:
vdo_df_groupby_cat

In [ ]:
vdo_df_groupby_cat.views

In [ ]:
vdo_df_groupby_cat.views.sum()

In [ ]:
vdo_df_groupby_cat.views.mean()

In [ ]:
vdo_df_groupby_cat.views.describe()

### Accessing groups in groupby

In [ ]:
for name, group in vdo_df_groupby_cat:
    print(name)
    print('----')
    print(type(group))
    print(group.columns)

In [ ]:
vdo_df_groupby_cat.get_group('Comedy')

### Like statistics of â€œMusicâ€ category over time?

In [ ]:
music_cat = vdo_df_withcat[vdo_df_withcat.category == 'Music']

In [ ]:
music_groupby_trending_date = music_cat.groupby('trending_dt')

In [ ]:
music_groupby_trending_date

In [ ]:
music_count_bydate = music_groupby_trending_date.video_id.count()

In [ ]:
music_count_bydate

In [ ]:
music_count_bydate.plot()

In [ ]:
music_groupby_trending_date.likes.sum()

In [ ]:
music_groupby_trending_date.likes.sum().plot()